In [3]:
from common_immunogit import *

/Users/guillaume.souede/PycharmProjects/immunogit
Structure Ok.


In [1]:
def json_metadata_extractor(json_data):
    extracted_data = {}

    # Extract relevant metadata from a JSON structure
    for model_id, model_data in json_data.items():
        extracted_data[model_id] = {
            "name": model_data.get("name"),
            "description": model_data.get("description"),
            "format": model_data.get("format", {}).get("name"),
            "publication": {
                "title": model_data.get("publication", {}).get("title"),
                "journal": model_data.get("publication", {}).get("journal"),
                "year": model_data.get("publication", {}).get("year"),
                "authors": [
                    {
                        "name": author.get("name"),
                        "institution": author.get("institution"),
                        "orcid": author.get("orcid")
                    }
                    for author in model_data.get("publication", {}).get("authors", [])
                ],
                # "link": model_data.get("publication", {}).get("link")
            },
            "files": model_data.get("files", {}).get("main", []),
            "contributors": model_data.get("contributors", {}),
            "tags": model_data.get("modelTags", [])
        }

    return extracted_data


def process_multiple_json_files(input_folder, delete=False):
    """Process JSON files and create separate metadata files for models."""

    models_file = os.path.join(md_path, "models_metadata.json")
    biomodels_file = os.path.join(md_path, "metadata_BioModels.json")
    reactome_file = os.path.join(md_path, "metadata_Reactome.json")

    merged_data = {"models": {}}
    biomodels_data = {}
    reactome_data = {}

    for filename in os.listdir(input_folder):
        if filename.endswith(".json") and not filename.startswith("."):  # Skip hidden files
            file_path = os.path.join(input_folder, filename)
            with open(file_path, "r") as file:
                json_data = json.load(file)
                metadata = json_metadata_extractor(json_data)

                merged_data["models"].append(metadata)

                if "BioModels" in filename:
                    biomodels_data.append(metadata)
                elif "Reactome" in filename:
                    reactome_data.append(metadata)

    # merged model
    with open(models_file, "w") as outfile:
        json.dump(merged_data, outfile, indent=4)
    print(f"Created {models_file}")

    # BioModels
    with open(biomodels_file, "w") as outfile:
        json.dump({"BioModels": biomodels_data}, outfile, indent=4)
    print(f"Created {biomodels_file}")

    # Reactome
    with open(reactome_file, "w") as outfile:
        json.dump({"Reactome": reactome_data}, outfile, indent=4)
    print(f"Created {reactome_file}")

    # Delete source files if True
    if delete:
        for filename in os.listdir(input_folder):
            file_path = os.path.join(input_folder, filename)
            if filename.endswith(".json") and not filename.startswith("."):
                os.remove(file_path)
        print(f"Source files deleted from {input_folder}.")
    else:
        print(f"{BLUE}Source files not deleted. Set delete = True to remove them.{RESET}")

def shrink_json_info(input_path):
    with open(input_path, 'r') as f:
        json_data = json.load(f)
    filtered_data = json_metadata_extractor(json_data)
    output_path = input_path.with_name(input_path.stem + "_filtered.json")
    with open(output_path, 'w') as f:
        json.dump(filtered_data, f, indent=4)
    print(f"Filtered metadata saved as : {output_path}")

In [ ]:
# process_multiple_json_files("../metadata/Raw", delete=False)

In [4]:
infile = md_path / "model_metadata.json"
shrink_json_info(infile)

Filtered metadata saved as : /Users/guillaume.souede/PycharmProjects/immunogit/metadata/model_metadata_filtered.json
